# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
csv_path = os.path.join('..','WeatherPy', 'output_data', 'cities.csv')
vac_df = pd.read_csv(csv_path).reset_index(drop=True).drop(['Unnamed: 0'], axis=1)
vac_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %
0,Jamestown,US,42.10,-79.24,73.40,53,3.36,1
1,Mar del Plata,AR,-38.00,-57.56,48.00,81,9.17,88
2,Severo-Kuril'sk,RU,50.68,156.12,56.64,89,9.13,97
3,Mataura,NZ,-46.19,168.86,51.80,79,1.86,100
4,Bredasdorp,ZA,-34.53,20.04,46.40,93,5.82,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
locations = vac_df[['Latitude', 'Longitude']]
humidity = vac_df['Humidity %']

In [59]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                              dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [66]:
ideal_df = vac_df.loc[(vac_df['Max Temp (F)'] <= 75) & (vac_df['Max Temp (F)'] >= 60) 
                      & (vac_df['Cloudiness %'] > 90) & (vac_df['Wind (mph)'] < 5), :]

ideal_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %
62,Raja,SS,8.46,25.68,68.72,95,2.48,100
129,Kichera,RU,55.94,110.10,64.74,80,1.14,100
183,Bay Roberts,CA,47.60,-53.26,61.00,100,3.31,99
191,Kyra,RU,49.58,111.98,60.53,86,2.26,100
197,El Zapote,MX,21.07,-102.80,71.76,68,3.62,96
270,Pokhara,NP,28.23,83.98,72.25,85,0.54,99
323,Santander,ES,43.46,-3.80,73.99,83,4.70,98
346,Wawa,NG,9.90,4.42,73.98,95,4.65,98
354,Villa Bruzual,VE,9.33,-69.12,74.88,96,3.42,100
368,Zhangjiakou,CN,40.81,114.88,71.69,77,0.81,99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
